In [1]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np

In [2]:
def img_crop (image):
  height=image.shape[0]
  width=image.shape[1]
  h_lower=int(height/2+ (0.45*height))
  h_upper=int(height/2- (0.38*height))
  w_left=int(width/2-(0.3*width))
  w_right=int(width/2+(0.2*width))
  #print(h_upper)
  image = image[h_upper:h_lower, w_left:w_right, :]

  return image

In [3]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=30):
    name = video_path.split("/")[-1].split(".")[0].lower()
    subname=name[19:]
    
  
  
    print(name)
    print(subname)
    if (name.find("_bhu") != -1) :
          save_path = os.path.join(save_dir, "Bhujangasana",subname)  
    elif (name.find("_padam") != -1) or (name.find("_padma") != -1):
          save_path = os.path.join(save_dir, "Padamasana",subname)
    elif (name.find("_shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana",subname)
    elif (name.find("_trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana",subname)
    elif (name.find("_vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana",subname)
    elif (name.find("_tad") != -1):
        save_path = os.path.join(save_dir, "Tadasana",subname)
    else:
        print(subname)
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            image=img_crop(frame)
            cv2.imwrite(f"{save_path}/{idx}.png", image)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("Yoga_Vid_Collected/*")
    save_dir = "dataset"

    for path in video_paths:
        save_frame(path, save_dir, gap=30)

yoga_vid_collected\abhay_bhuj
abhay_bhuj
dataset\Bhujangasana\abhay_bhuj
yoga_vid_collected\abhay_padam
abhay_padam
dataset\Padamasana\abhay_padam
yoga_vid_collected\abhay_shav
abhay_shav
dataset\Shavasana\abhay_shav
yoga_vid_collected\abhay_tada
abhay_tada
dataset\Tadasana\abhay_tada
yoga_vid_collected\abhay_trik
abhay_trik
dataset\Trikonasana\abhay_trik
yoga_vid_collected\abhay_vriksh
abhay_vriksh
dataset\Vrikshasana\abhay_vriksh
yoga_vid_collected\ameya_bhuj
ameya_bhuj
dataset\Bhujangasana\ameya_bhuj
yoga_vid_collected\ameya_padam
ameya_padam
dataset\Padamasana\ameya_padam
yoga_vid_collected\ameya_shav
ameya_shav
dataset\Shavasana\ameya_shav
yoga_vid_collected\ameya_tada
ameya_tada
dataset\Tadasana\ameya_tada
yoga_vid_collected\ameya_trik
ameya_trik
dataset\Trikonasana\ameya_trik
yoga_vid_collected\ameya_vriksh
ameya_vriksh
dataset\Vrikshasana\ameya_vriksh
yoga_vid_collected\bhumi_bhuj
bhumi_bhuj
dataset\Bhujangasana\bhumi_bhuj
yoga_vid_collected\bhumi_padam
bhumi_padam
dataset\Pada

In [39]:
# Load the input image.
def image_to_movenet(path):
    
    if (path.find("Bhujangasana") != -1) :
          label="Bhujangasana"  
    elif (path.find("Padamasana") != -1):
          label="Padamasana"
    elif (path.find("Shavasana") != -1):
          label="Shavasana"
    elif (path.find("Trikonasana") != -1):
          label="Trikonasana"
    elif (path.find("Vrikshasana") != -1):
          label="Vrikshasana"
    elif (path.find("Tadasana") != -1):
          label="Tadasana"
        
    image_path = path
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)

    # Download the model from TF Hub.
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    movenet = model.signatures['serving_default']

    # Run model inference.
    outputs = movenet(image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints = outputs['output_0']
    keypoints_tup=(keypoints,label)
    keypts_labels.append(keypoints_tup)
    print(path)
    
    print(keypts_labels)
    
    

In [40]:
import os
import time

path_lst=[]
keypts_labels=[]
class_list=os.listdir('dataset')
for name in class_list:
    for image in os.listdir(f"dataset/{name}"):
        for item in os.listdir(f"dataset/{name}/{image}"):
#             add_tup=(os.path.abspath(f"dataset/{name}/{image}/{item}"),name)
            path_lst.append(os.path.abspath(f"dataset/{name}/{image}/{item}"))



path_lst = sorted(path_lst,key=os.path.getmtime)
for path in path_lst:
    image_to_movenet(path)
    


E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\dataset\Bhujangasana\abhay_bhuj\15.png
[(<tf.Tensor: shape=(1, 1, 17, 3), dtype=float32, numpy=
array([[[[0.78621894, 0.87021947, 0.50614953],
         [0.7801042 , 0.87738633, 0.4593199 ],
         [0.77937704, 0.87538415, 0.50933564],
         [0.7547425 , 0.86347234, 0.5314946 ],
         [0.75545913, 0.86047244, 0.5078999 ],
         [0.7497095 , 0.8122004 , 0.5055896 ],
         [0.7793317 , 0.81107634, 0.69365615],
         [0.7556962 , 0.6882595 , 0.255724  ],
         [0.7984366 , 0.6940263 , 0.56903315],
         [0.78506917, 0.6002404 , 0.2885837 ],
         [0.8078834 , 0.58478236, 0.49464354],
         [0.73355955, 0.57866436, 0.6360941 ],
         [0.74406546, 0.5759433 , 0.59036964],
         [0.7438633 , 0.43917832, 0.64149356],
         [0.77175796, 0.43098912, 0.42472148],
         [0.7200045 , 0.33020777, 0.3187891 ],
         [0.7472891 , 0.30311382, 0.38912213]]]], dtype=float32)>, 'Bhujangasana')]
E:\BE\FinalYearPro

E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\dataset\Bhujangasana\abhay_bhuj\135.png
[(<tf.Tensor: shape=(1, 1, 17, 3), dtype=float32, numpy=
array([[[[0.78621894, 0.87021947, 0.50614953],
         [0.7801042 , 0.87738633, 0.4593199 ],
         [0.77937704, 0.87538415, 0.50933564],
         [0.7547425 , 0.86347234, 0.5314946 ],
         [0.75545913, 0.86047244, 0.5078999 ],
         [0.7497095 , 0.8122004 , 0.5055896 ],
         [0.7793317 , 0.81107634, 0.69365615],
         [0.7556962 , 0.6882595 , 0.255724  ],
         [0.7984366 , 0.6940263 , 0.56903315],
         [0.78506917, 0.6002404 , 0.2885837 ],
         [0.8078834 , 0.58478236, 0.49464354],
         [0.73355955, 0.57866436, 0.6360941 ],
         [0.74406546, 0.5759433 , 0.59036964],
         [0.7438633 , 0.43917832, 0.64149356],
         [0.77175796, 0.43098912, 0.42472148],
         [0.7200045 , 0.33020777, 0.3187891 ],
         [0.7472891 , 0.30311382, 0.38912213]]]], dtype=float32)>, 'Bhujangasana'), (<tf.Tensor: shap

E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\dataset\Bhujangasana\abhay_bhuj\195.png
[(<tf.Tensor: shape=(1, 1, 17, 3), dtype=float32, numpy=
array([[[[0.78621894, 0.87021947, 0.50614953],
         [0.7801042 , 0.87738633, 0.4593199 ],
         [0.77937704, 0.87538415, 0.50933564],
         [0.7547425 , 0.86347234, 0.5314946 ],
         [0.75545913, 0.86047244, 0.5078999 ],
         [0.7497095 , 0.8122004 , 0.5055896 ],
         [0.7793317 , 0.81107634, 0.69365615],
         [0.7556962 , 0.6882595 , 0.255724  ],
         [0.7984366 , 0.6940263 , 0.56903315],
         [0.78506917, 0.6002404 , 0.2885837 ],
         [0.8078834 , 0.58478236, 0.49464354],
         [0.73355955, 0.57866436, 0.6360941 ],
         [0.74406546, 0.5759433 , 0.59036964],
         [0.7438633 , 0.43917832, 0.64149356],
         [0.77175796, 0.43098912, 0.42472148],
         [0.7200045 , 0.33020777, 0.3187891 ],
         [0.7472891 , 0.30311382, 0.38912213]]]], dtype=float32)>, 'Bhujangasana'), (<tf.Tensor: shap

E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\dataset\Bhujangasana\abhay_bhuj\255.png
[(<tf.Tensor: shape=(1, 1, 17, 3), dtype=float32, numpy=
array([[[[0.78621894, 0.87021947, 0.50614953],
         [0.7801042 , 0.87738633, 0.4593199 ],
         [0.77937704, 0.87538415, 0.50933564],
         [0.7547425 , 0.86347234, 0.5314946 ],
         [0.75545913, 0.86047244, 0.5078999 ],
         [0.7497095 , 0.8122004 , 0.5055896 ],
         [0.7793317 , 0.81107634, 0.69365615],
         [0.7556962 , 0.6882595 , 0.255724  ],
         [0.7984366 , 0.6940263 , 0.56903315],
         [0.78506917, 0.6002404 , 0.2885837 ],
         [0.8078834 , 0.58478236, 0.49464354],
         [0.73355955, 0.57866436, 0.6360941 ],
         [0.74406546, 0.5759433 , 0.59036964],
         [0.7438633 , 0.43917832, 0.64149356],
         [0.77175796, 0.43098912, 0.42472148],
         [0.7200045 , 0.33020777, 0.3187891 ],
         [0.7472891 , 0.30311382, 0.38912213]]]], dtype=float32)>, 'Bhujangasana'), (<tf.Tensor: shap

E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\dataset\Bhujangasana\abhay_bhuj\315.png
[(<tf.Tensor: shape=(1, 1, 17, 3), dtype=float32, numpy=
array([[[[0.78621894, 0.87021947, 0.50614953],
         [0.7801042 , 0.87738633, 0.4593199 ],
         [0.77937704, 0.87538415, 0.50933564],
         [0.7547425 , 0.86347234, 0.5314946 ],
         [0.75545913, 0.86047244, 0.5078999 ],
         [0.7497095 , 0.8122004 , 0.5055896 ],
         [0.7793317 , 0.81107634, 0.69365615],
         [0.7556962 , 0.6882595 , 0.255724  ],
         [0.7984366 , 0.6940263 , 0.56903315],
         [0.78506917, 0.6002404 , 0.2885837 ],
         [0.8078834 , 0.58478236, 0.49464354],
         [0.73355955, 0.57866436, 0.6360941 ],
         [0.74406546, 0.5759433 , 0.59036964],
         [0.7438633 , 0.43917832, 0.64149356],
         [0.77175796, 0.43098912, 0.42472148],
         [0.7200045 , 0.33020777, 0.3187891 ],
         [0.7472891 , 0.30311382, 0.38912213]]]], dtype=float32)>, 'Bhujangasana'), (<tf.Tensor: shap

E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\dataset\Bhujangasana\abhay_bhuj\345.png
[(<tf.Tensor: shape=(1, 1, 17, 3), dtype=float32, numpy=
array([[[[0.78621894, 0.87021947, 0.50614953],
         [0.7801042 , 0.87738633, 0.4593199 ],
         [0.77937704, 0.87538415, 0.50933564],
         [0.7547425 , 0.86347234, 0.5314946 ],
         [0.75545913, 0.86047244, 0.5078999 ],
         [0.7497095 , 0.8122004 , 0.5055896 ],
         [0.7793317 , 0.81107634, 0.69365615],
         [0.7556962 , 0.6882595 , 0.255724  ],
         [0.7984366 , 0.6940263 , 0.56903315],
         [0.78506917, 0.6002404 , 0.2885837 ],
         [0.8078834 , 0.58478236, 0.49464354],
         [0.73355955, 0.57866436, 0.6360941 ],
         [0.74406546, 0.5759433 , 0.59036964],
         [0.7438633 , 0.43917832, 0.64149356],
         [0.77175796, 0.43098912, 0.42472148],
         [0.7200045 , 0.33020777, 0.3187891 ],
         [0.7472891 , 0.30311382, 0.38912213]]]], dtype=float32)>, 'Bhujangasana'), (<tf.Tensor: shap

KeyboardInterrupt: 